<a href="https://colab.research.google.com/github/farroshsy/MidtermExam_BigData/blob/main/FPGrowth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Midterm Exam**
## Spark Streaming and Reccomendation Systems
---

Name: Farros Hilmi Syafei 
<br>
Student ID: 5025201012
<br>
Class: Big Data A
<br>
Lecturer: Abdul Munif, S.Kom., M.Sc.


## Problem
### 2. Recommendation Systems - Frequent Pattern Mining
You are given the market basket dataset (market-basket.csv). 
*   Use the FP-growth algorithm in Apache Spark to find the most frequent items. Save
your result into an .xlsx files.
*   Do the experiment with different minSupport and minConfidence values. Give the conclusion at the end of experiment.



# Data Driven Dealings Development

- EDA on Sales Data
- RFM Clustering
- Predicting Sales
- Market Basket Analysis
- Recommending Items per Customer

## Market Basket Analysis

In [16]:
%%capture
!sudo apt-get update --fix-missing

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
#!wget -q https://downloads.apache.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

!mv spark-3.0.0-bin-hadoop3.2.tgz sparkkk
!tar xf sparkkk
!pip install -q findspark

In [17]:
!pip install spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58760 sha256=575c87df418b36c398b79223c43a327f15b3feb8774b265fe870dfa4f07016c8
  Stored in directory: /root/.cache/pip/wheels/bc/0f/6c/b41528ca0fd4d46513185e90da91bc1c484bba6138641c5a62
Successfully built spark


In [27]:
# Sets up the environment variables for Java and Spark, initializes the findspark library
# Creates a new SparkSession for a PySpark application named 'fpgrowth'.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName('fpgrowth') \
    .getOrCreate()

spark   

In [26]:
# To be able to use your data stored in your Google Drive you first need to mount your Google Drive so you can load and save files to it. 
from google.colab import drive
drive.mount('/content/gdrive')
#You'll need to put in a token which Google will generate for you as soon as you click on the link

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [41]:
import csv
csv_path = '/content/gdrive/MyDrive/BigData/market-basket.csv'
with open(csv_path, 'r') as f:
    temp_lines = f.readline() + '\n' + f.readline()
    # The csv.Sniffer() method is used to detect the delimiter used in the file (either ; or ,) and then the dialect object is used to read the file with the correct delimiter. The pandas.read_csv() method is used to load the data into a pandas DataFrame object called data.
    dialect = csv.Sniffer().sniff(temp_lines, delimiters=';,')
    f.seek(0)
    data = pd.read_csv(f, dialect=dialect, error_bad_lines=False)
data.head()

<ipython-input-41-cb59a4411cf1>:7: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(f, dialect=dialect, error_bad_lines=False)
<ipython-input-41-cb59a4411cf1>:7: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f, dialect=dialect, error_bad_lines=False)


,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 08:26,"2,55",17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 08:26,"2,75",17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom


In [49]:
from google.colab import files
from pyspark.sql import functions as F
from pyspark.ml.fpm import FPGrowth

data = data.astype(str)
data['BillNo'] = data['BillNo'].astype(str)

sparkdata = spark.createDataFrame(data)
basketdata = sparkdata.dropDuplicates(['BillNo', 'Itemname']).sort('BillNo')
basketdata = basketdata.groupBy("BillNo").agg(F.collect_list("Itemname")).sort('BillNo')

#### First Experiment
minSupport = 0.002 and minConfidence == 0.002

In [ ]:
#Frequent Pattern Growth – FP Growth is a method of mining frequent itemsets
fpGrowth = FPGrowth(itemsCol="collect_list(Itemname)", minSupport=0.002, minConfidence=0.002) 
model = fpGrowth.fit(basketdata)

# Display frequent itemsets.
model.freqItemsets.show()
items = model.freqItemsets
# Display generated association rules.
model.associationRules.show()
rules = model.associationRules
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(basketdata).show()
transformed = model.transform(basketdata)

+--------------------+----+
|               items|freq|
+--------------------+----+
|[FELTCRAFT BUTTER...| 471|
|[FELTCRAFT BUTTER...|  56|
|[FELTCRAFT BUTTER...|  59|
|[FELTCRAFT BUTTER...| 124|
|[FELTCRAFT BUTTER...|  79|
|[FELTCRAFT BUTTER...|  48|
|[FELTCRAFT BUTTER...|  44|
|[FELTCRAFT BUTTER...| 127|
|[FELTCRAFT BUTTER...|  49|
|[FELTCRAFT BUTTER...|  55|
|[FELTCRAFT BUTTER...|  47|
|[FELTCRAFT BUTTER...|  47|
|[FELTCRAFT BUTTER...|  48|
|[FELTCRAFT BUTTER...|  73|
|[FELTCRAFT BUTTER...|  49|
|[FELTCRAFT BUTTER...|  70|
|[FELTCRAFT BUTTER...|  54|
|[FELTCRAFT BUTTER...|  52|
|[FELTCRAFT BUTTER...|  46|
|[FELTCRAFT BUTTER...|  57|
+--------------------+----+
only showing top 20 rows



In [ ]:
# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = items.select("*").toPandas()
result_pdf.head()

In [ ]:
result_pdf.to_excel('result_pdfItemsFreq_Exp1.xlsx')

In [ ]:
rules_pdf = rules.select("*").toPandas()
rules_pdf.head()

,antecedent,consequent,confidence,lift
0,[128],[67],0.337998,8.287536
1,[128],[91],0.342304,10.666919
2,[128],[104],0.387513,7.764057
3,[128],[92],0.315393,9.648273
4,[128],[103],0.355221,8.102645


In [ ]:
rules_pdf.to_excel('rules_pdfAnteConseConfLift_Exp1.xlsx')

In [ ]:
transformed_pdf = transformed.select("*").toPandas()
transformed_pdf.head()

,SalesTransactionID,collect_list(SalesItem),prediction
0,0,[0],[]
1,1,"[0, 1, 2]",[]
2,2,[1],[]
3,3,[0],[]
4,4,[0],[]


In [ ]:
transformed_pdf.to_excel('transformed_pdfSalesTransactionIDCollectListPred_Exp1.xlsx')

#### Second Experiment
minSupport = 0.005 and minConfidence == 0.005

In [ ]:
#Frequent Pattern Growth – FP Growth is a method of mining frequent itemsets
fpGrowth = FPGrowth(itemsCol="collect_list(Itemname)", minSupport=0.005, minConfidence=0.005) 
model = fpGrowth.fit(basketdata)

# Display frequent itemsets.
model.freqItemsets.show()
items = model.freqItemsets
# Display generated association rules.
model.associationRules.show()
rules = model.associationRules
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(basketdata).show()
transformed = model.transform(basketdata)

In [ ]:
# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = items.select("*").toPandas()
result_pdf.head()

In [ ]:
result_pdf.to_excel('result_pdfItemsFreq_Exp2.xlsx')

In [ ]:
rules_pdf = rules.select("*").toPandas()
rules_pdf.head()

In [ ]:
rules_pdf.to_excel('rules_pdfAnteConseConfLift_Exp1.xlsx')

In [ ]:
transformed_pdf = transformed.select("*").toPandas()
transformed_pdf.head()

In [ ]:
transformed_pdf.to_excel('transformed_pdfSalesTransactionIDCollectListPred_Exp1.xlsx')

#### Third Experiment
minSupport = 0.01 and minConfidence == 0.01

In [ ]:
#Frequent Pattern Growth – FP Growth is a method of mining frequent itemsets
fpGrowth = FPGrowth(itemsCol="collect_list(Itemname)", minSupport=0.01, minConfidence=0.01) 
model = fpGrowth.fit(basketdata)

# Display frequent itemsets.
model.freqItemsets.show()
items = model.freqItemsets
# Display generated association rules.
model.associationRules.show()
rules = model.associationRules
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(basketdata).show()
transformed = model.transform(basketdata)

In [ ]:
# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = items.select("*").toPandas()
result_pdf.head()

In [ ]:
result_pdf.to_excel('result_pdfItemsFreq_Exp3.xlsx')

In [ ]:
rules_pdf = rules.select("*").toPandas()
rules_pdf.head()

In [ ]:
rules_pdf.to_excel('rules_pdfAnteConseConfLift_Exp1.xlsx')

In [ ]:
transformed_pdf = transformed.select("*").toPandas()
transformed_pdf.head()

In [ ]:
transformed_pdf.to_excel('transformed_pdfSalesTransactionIDCollectListPred_Exp1.xlsx')

## Conclusion



*   Minimum Support: This parameter determines the minimum frequency that an itemset must have in order to be considered frequent. Itemsets that occur less frequently than the minimum support threshold are pruned, as they are unlikely to be interesting or informative. Increasing the minimum support value leads to fewer frequent itemsets being generated, while decreasing it leads to more frequent itemsets.
*   Minimum Confidence: This parameter determines the minimum level of confidence that an association rule must have in order to be considered interesting or informative. Association rules that have less confidence than the minimum confidence threshold are considered uninteresting and are pruned. Increasing the minimum confidence value leads to fewer association rules being generated, while decreasing it leads to more association rules.

Sure! In general, the minimum support and minimum confidence parameters in association rule mining algorithms such as FP-growth can have a significant impact on the results generated.

Minimum Support: This parameter determines the minimum frequency that an itemset must have in order to be considered frequent. Itemsets that occur less frequently than the minimum support threshold are pruned, as they are unlikely to be interesting or informative. Increasing the minimum support value leads to fewer frequent itemsets being generated, while decreasing it leads to more frequent itemsets.

Minimum Confidence: This parameter determines the minimum level of confidence that an association rule must have in order to be considered interesting or informative. Association rules that have less confidence than the minimum confidence threshold are considered uninteresting and are pruned. Increasing the minimum confidence value leads to fewer association rules being generated, while decreasing it leads to more association rules.

In general, finding the optimal values for these parameters requires a balance between generating enough frequent itemsets or association rules to be useful and avoiding overfitting or generating too many irrelevant results. This can be achieved through iterative experimentation with different parameter values, as well as domain knowledge and intuition.

